# Freesound General Purpose Audio Tagging Submission Notebook
### Graham Bachman

This notebook is just the code used to generate predictions on the test set of spectrogram images for the kaggle competiton. 

## Load Data

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

import keras
from keras_preprocessing.image import ImageDataGenerator

## Create Dataframes

In [ ]:
train = pd.read_csv('../input/freesound-audio-tagging/train.csv')
train['image_file'] = train.fname.apply(lambda x : x[:-4] + '.jpg')
print(train.shape)
train.head()

In [ ]:
test = pd.read_csv('../input/freesound-audio-tagging/test_post_competition.csv')
test['image_file'] = test.fname.apply(lambda x : x[:-4] + '.jpg')
print(test.shape)
test.head()

In [ ]:
test_public = test.query('label != "None"').copy()
print(test_public.shape)
test_public.head()

## Data Generator

In [ ]:
test_public_path = '../input/freesound-2018-100dpi/labeled_test_images'
test_path = '../input/freesound-2018-100dpi/full_test_images'

print(len(os.listdir(test_public_path)))
print(len(os.listdir(test_path)))

In [ ]:
bs = 64
target_size = (50, 75)

public_test_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

public_test_gen = public_test_datagen.flow_from_dataframe(
    dataframe = test_public,
    directory = test_public_path,
    x_col = 'image_file',
    y_col = 'label',
    batch_size = bs,
    seed = 1,
    shuffle = False,
    class_mode = 'categorical',
    target_size = target_size
)

test_gen = test_datagen.flow_from_dataframe(
    dataframe = test,
    directory = test_path,
    x_col = 'image_file',
    y_col = None,
    batch_size = bs,
    seed = 1,
    shuffle = False,
    class_mode = None,
    target_size = target_size
)

In [ ]:
test_batches = len(test_gen)
public_test_batches = len(public_test_gen)

print(test_batches)
print(public_test_batches)

In [ ]:
label_to_index = public_test_gen.class_indices
index_to_label = {v:k for k,v in label_to_index.items()}
print(label_to_index)

## Load and summarize our best model

In [ ]:
new_model = keras.models.load_model('../input/models/freesound_model_v03_50.h5')
new_model.summary()

## Make predictions

In [ ]:
%%time 

valid_probs = new_model.predict(public_test_gen, steps=public_test_batches)

print(valid_probs.shape)

## Mean Average Precision
Mean Average Precision (MAP) is the metric used in this kaggle competition for assessing submissions. 
It accepts three ranked predicted labels for each audio clip. Full credit is awarded if the correct label is listed first, while lesser credit is awarded where the correct label is the second or third highest probability. 
### Define function to obtain top 3 highest class probabilities

In [ ]:
def get_top_3(p):
    top_classes = np.argpartition(p, -3)[-3:]                  # Gives top 3 classes in increasing order
    top_classes = top_classes[np.argsort(p[top_classes])]      # Sorts in increasing order
    top_classes = np.flip(top_classes)                         # Flips the order.

    top_probs = p[top_classes]              
    
    return top_probs, top_classes

t3p, t3c = get_top_3(valid_probs[0,:])
print(t3p.round(3))
print(t3c)

### Visualize

In [ ]:
N_train = valid_probs.shape[0]
top_3_probs = np.zeros(shape=(N_train, 3))

for i in range(N_train):    
    top_probs, top_classes = get_top_3(valid_probs[i, :])
    top_3_probs[i,:] = top_probs

plt.figure(figsize=[10,6])
for i in range(3):
    plt.subplot(3,1,i+1)
    plt.hist(top_3_probs[:,i], color='lightsalmon', edgecolor='k', bins = np.arange(0, 1.01, 0.025))
    plt.title(f'Top {i+1} Probabilities')
    plt.yscale('log')
plt.tight_layout()
plt.show()

In [ ]:
enc_labels = test_public.label.apply(lambda x : label_to_index[x]).values
print(enc_labels)

In [ ]:
def MAP3(probs, labels, t):
    N = len(labels)
    top_3_probs = np.zeros(shape=(N, 3))
    sum_ap3 = 0

    for i in range(N):
        top_probs, top_classes = get_top_3(probs[i, :])

        # Keep Probs Over Threshold
        sel = top_probs > t
        sel[0] = True                               # Always keep first pred

        top_classes = top_classes[sel]

        K = len(top_classes)   # Number of classes to submit
        if K == 3:
            scores = np.array([11/18, 5/18, 2/18])
        elif K == 2:
            scores = np.array([3/4, 1/4])
        else:
            scores = np.array([1])

        sel = (top_classes == labels[i])

        #print(f'{str(top_classes):<20}{labels[i]:<6}{sel}')

        ap3 = np.sum(scores * sel)

        sum_ap3 += ap3

    map3 = sum_ap3 / len(labels)

    return map3

MAP3(valid_probs, enc_labels, 0.5)

## Tune Threshold

In [ ]:
MAP3_scores = []

t_array = np.arange(0.05, 1.01, 0.05) 

for t in t_array:
    MAP3_scores.append(MAP3(valid_probs, enc_labels, t))
    
plt.plot(t_array, MAP3_scores)
plt.scatter(t_array, MAP3_scores)
plt.show()

## Make predictions on the test set and create submission file

In [ ]:
%%time 

test_probs = new_model.predict(test_gen, steps=test_batches)

print(test_probs.shape)

In [ ]:
submission = pd.read_csv('../input/freesound-audio-tagging/sample_submission.csv')
submission.head()

In [ ]:
np.sum(submission.fname == test.fname)
print(test_probs[:5, :5])

In [ ]:
t = 0.2

for i in range(len(submission)):
#for i in range(10):
    
    # Get top 3 classes
    top_probs, top_classes = get_top_3(test_probs[i, :])
    
    # Select only those above threshold
    sel = top_probs > t
    sel[0] = True
    sel_classes = top_classes[sel]
    
    pred_labels = [index_to_label[i] for i in sel_classes]
    pred_string = ' '.join(pred_labels)
    submission.loc[i, 'label'] = pred_string
    
submission.head(20)

In [ ]:
submission.to_csv('submission_02.csv', index=False)